# Topological Exploratory Data Analysis

Mathieu Carrière, https://mathieucarriere.github.io/website/

In [ ]:
from gudhi.cover_complex import MapperComplex, GraphInducedComplex, NerveComplex
from gudhi import bottleneck_distance
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

In this notebook, we will see how to use Gudhi in order to perform topological dimension reduction: we will compute simplicial complex approximations of point cloud / distance matrices. These complexes will be either [Mapper complexes](https://diglib.eg.org/handle/10.2312/SPBG.SPBG07.091-100) or [Graph Induced complexes](http://web.cse.ohio-state.edu/~dey.8/GIC/gic.html). Both complexes use covers of the initial space (such as Voronoi partitions or preimages of filter functions), and use these covers to generate simplicial complexes, either by taking the nerve (Mapper) or by checking the presence of colored cliques (Graph Induced). 

Gudhi can handle both point clouds and distance matrices. Let's start with a point cloud.

# Point cloud

Load an example point cloud.

In [ ]:
X = np.loadtxt('datasets/human2.txt')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[::2,1], X[::2,0], X[::2,2], s=1)
limits = np.array([ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()])
origin = np.mean(limits, axis=1)
radius = 0.5 * np.max(np.abs(limits[:, 1] - limits[:, 0]))
x, y, z = origin
ax.set_xlim3d([x - radius, x + radius])
ax.set_ylim3d([y - radius, y + radius])
ax.set_zlim3d([z - radius, z + radius])
plt.show()

We will use the height function to color the complex nodes.

In [ ]:
height = X[:,2]

We next provide different configurations for computing cover complexes:

Graph Induced complex with a Voronoi partition with 100 randomly sampled germs and Rips graph obtained with automatic threshold. 

In [ ]:
cover_complex = GraphInducedComplex(
    input_type='point cloud', cover='voronoi', colors=height, mask=0,
    graph="rips", rips_threshold=None, N=100, beta=0., C=10,
    voronoi_samples=100, 
    input_name="human", cover_name="voronoi", color_name="coord2", verbose=True)

Graph Induced complex with a preimage partition with automatic resolution and Rips graph obtained with automatic threshold. 

In [ ]:
cover_complex = GraphInducedComplex(
    input_type='point cloud', cover='functional', colors=height, mask=0,
    graph="rips", rips_threshold=None, N=100, beta=0., C=10,
    filter=height, resolution=None, gain=0.,
    input_name="human", cover_name="coord2", color_name="coord2", verbose=True)

Mapper complex with a preimage cover with automatic resolution and hierarchical clustering obtained with automatic threshold. 

In [ ]:
cover_complex = MapperComplex(
    input_type='point cloud', colors=height[:,np.newaxis], mask=0,
    clustering=None, N=100, beta=0., C=10,
    filters=height[:,np.newaxis], filter_bnds=None, resolutions=None, gains=None,
    input_name="human", cover_name="coord2", color_name="coord2", verbose=True)

Mapper complex with a preimage cover with automatic resolution from a 2D function and hierarchical clustering obtained with automatic threshold. 

In [ ]:
filt2d = np.hstack([height[:,np.newaxis],X[:,0:1]])

In [ ]:
cover_complex = MapperComplex(
    input_type='point cloud', colors=filt2d, mask=0,
    clustering=None, N=100, beta=0., C=10,
    filters=filt2d, filter_bnds=None, resolutions=[20,2], gains=None,
    input_name="human", cover_name="coord2", color_name="coord2", verbose=True)

# Distance matrix

We can actually process the dataset using only the pairwise distances between points.

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
X = pairwise_distances(X)

In [ ]:
plt.figure()
plt.imshow(X)
plt.show()

This time, color is given by eccentricity.

In [ ]:
ecc = X.max(axis=0)

In [ ]:
cover_complex = GraphInducedComplex(
    input_type='distance matrix', cover='functional', colors=ecc, mask=0,
    graph="rips", rips_threshold=None, N=100, beta=0., C=10,
    filter=ecc, resolution=None, gain=0.,
    input_name="human", cover_name="coord2", color_name="coord2", verbose=True)

In [ ]:
cover_complex = GraphInducedComplex(
    input_type='distance matrix', cover='voronoi', colors=ecc, mask=0,
    graph="rips", rips_threshold=None, N=100, beta=0., C=10,
    voronoi_samples=100, 
    input_name="human", cover_name="coord2", color_name="coord2", verbose=True)

In [ ]:
cover_complex = MapperComplex(
    input_type='distance matrix', colors=ecc[:,np.newaxis], mask=0,
    clustering=None, N=100, beta=0., C=10,
    filters=ecc[:,np.newaxis], filter_bnds=None, resolutions=None, gains=None,
    input_name="human", cover_name="coord2", color_name="coord2", verbose=True)

# Complex computation

The cover complex can now be computed in a single line of code!

In [ ]:
_ = cover_complex.fit(X)

# Visualization

You can visualize the complex in three different ways with Gudhi.

1. You can use Python package `networkx`.

In [ ]:
import networkx as nx

In [ ]:
G = cover_complex.get_networkx()

In [ ]:
plt.figure()
nx.draw(G, pos=nx.kamada_kawai_layout(G), node_color=[cover_complex.node_info[v]["colors"][0] for v in G.nodes()])
plt.show()

2. You can create a DOT file that can be processed later with `neato` to produce a PDF.

In [ ]:
cover_complex.print_to_dot()

In [ ]:
!neato -Tpdf human.dot -o human.pdf 

3. You can create a TXT file that you can process later with our KeplerMapper wrapper to produce a HTML file that you can visualize in browser.

In [ ]:
cover_complex.print_to_txt()

In [ ]:
!python /home/mcarrier/Git/gudhi/src/Nerve_GIC/utilities/KeplerMapperVisuFromTxtFile.py -f human.txt

# Topological features

In this section, we will do a variety of post-processing, statistical tasks for the computed cover complex, including, e.g., computing the topological features and/or assessing their stability. For these operations, we will need a set of helper functions, which we define below.

In [ ]:
cover_complex.data = X

In [ ]:
from scipy.sparse.csgraph    import dijkstra, shortest_path, connected_components
from scipy.stats             import ks_2samp

def find(i, parents):
    if parents[i] == i:
        return i
    else:
        return find(parents[i], parents)

def union(i, j, parents, f):
    if f[i] <= f[j]:
        parents[j] = i
    else:
        parents[i] = j
        
def compute_topological_features(M, threshold=0.):
    """
    Compute the topological features (connected components, up/down branches, loops) of the 1-skeleton of the cover complex. Connected components and loops are computed with scipy functions, and branches are detected with Union-Find and 0-dimensional persistence of the 1-skeleton.
    Parameters:
        threshold (float): any topological feature whose size is less than this parameter (relative to the first color function) will be discarded.
    Returns:
        dgm (list of (dim,(a,b)) tuples): list of feature characteristics. dim is the topological dimension of the feature (0 for CCs and branches, 1 for loops), a,b are the min and max of the first color function along the feature.
        bnds (list of lists): list of feature points. Each element of this list is the list of point IDs forming the corresponding feature. 
    """
    st = M.simplex_tree
    num_nodes = st.num_vertices()
    function, namefunc, invnamefunc = {}, {}, {}
    nodeID = 0
    for (s,_) in st.get_skeleton(0):
        namefunc[s[0]] = nodeID
        invnamefunc[nodeID] = s[0]
        function[s[0]] = M.node_info[s[0]]["colors"][0]
        nodeID += 1
    dgm, bnd = [], []

    # connected_components
    A = np.zeros([num_nodes, num_nodes])
    for (splx,_) in st.get_skeleton(1):
        if len(splx) == 2:
            A[namefunc[splx[0]], namefunc[splx[1]]] = 1
            A[namefunc[splx[1]], namefunc[splx[0]]] = 1
    _, ccs = connected_components(A, directed=False)
    for ccID in np.unique(ccs):
        pts = np.argwhere(ccs == ccID).flatten()
        vals = [function[invnamefunc[p]] for p in pts]
        if np.abs(min(vals) - max(vals)) >= threshold:
            dgm.append((0, (min(vals), max(vals))))
            bnd.append([invnamefunc[p] for p in pts])

    # loops
    G = M.get_networkx()
    try:
        from networkx import cycle_basis
        bndall = cycle_basis(G)
        for pts in bndall:
            vals = [function[p] for p in pts]
            if np.abs(min(vals) - max(vals)) >= threshold:	
                dgm.append((1,(min(vals), max(vals))))
                bnd.append(pts)
    except ImportError:
        print("Networkx not found, loops not computed")
        
    # branches
    for topo_type in ["downbranch", "upbranch"]:

        lfunction = []
        for i in range(num_nodes):
            lfunction.append(function[invnamefunc[i]])

        # upranch is downbranch of opposite function
        if topo_type == "upbranch":
            lfunction = [-f for f in lfunction]

        # sort vertices according to function values and compute inverse function 
        sorted_idxs = np.argsort(np.array(lfunction))
        inv_sorted_idxs = np.zeros(num_nodes)
        for i in range(num_nodes):
            inv_sorted_idxs[sorted_idxs[i]] = i

        # go through all vertices in ascending function order
        persistence_diag, persistence_set, parents, visited = {}, {}, -np.ones(num_nodes, dtype=np.int32), {}
        for i in range(num_nodes):

            current_pt = sorted_idxs[i]
            neighbors = np.ravel(np.argwhere(A[current_pt,:] == 1))
            lower_neighbors = [n for n in neighbors if inv_sorted_idxs[n] <= i] if len(neighbors) > 0 else []

            # no lower neighbors: current point is a local minimum
            if lower_neighbors == []:
                parents[current_pt] = current_pt

            # some lower neighbors exist
            else:

                # find parent pg of lower neighbors with lowest function value
                neigh_parents = [find(n, parents) for n in lower_neighbors]
                pg = neigh_parents[np.argmin([lfunction[n] for n in neigh_parents])]

                # set parent of current point to pg
                parents[current_pt] = pg

                # for each lower neighbor, we will create a persistence diagram point and corresponding set of nodes
                for neighbor in lower_neighbors:

                    # get parent pn
                    pn = find(neighbor, parents)
                    val = lfunction[pn]
                    persistence_set[pn] = []

                    # we will create persistence set only if parent pn is not local minimum pg
                    if pn != pg:
                        # go through all strictly lower nodes with parent pn
                        for v in sorted_idxs[:i]:
                            if find(v, parents) == pn:
                                # if it is already part of another persistence set, continue
                                try:
                                    visited[v]
                                # else, mark visited and include it in current persistence set
                                except KeyError:
                                    visited[v] = True
                                    persistence_set[pn].append(v)

                        # add current point to persistence set
                        persistence_set[pn].append(current_pt)

                        # do union and create persistence point corresponding to persistence set if persistence is sufficiently large
                        if np.abs(lfunction[pn]-lfunction[current_pt]) >= threshold:
                            persistence_diag[pn] = current_pt
                            union(pg, pn, parents, lfunction)

        for key, val in iter(persistence_diag.items()):
            if topo_type == "downbranch":
                dgm.append((0, (lfunction[key],  lfunction[val])))
            elif topo_type == "upbranch":
                dgm.append((0, (-lfunction[val], -lfunction[key])))
            bnd.append([invnamefunc[v] for v in persistence_set[key]])

    bnd = [list(b) for b in bnd]
    M.persistence_diagram, M.persistence_sets = dgm, bnd 
    return dgm, bnd

def bootstrap_topological_features(M, N):
    """
    Use bootstrap to empirically assess stability of the features. This function computes a distribution of bottleneck distances, that can used afterwards to run tests on each topological feature.
    Parameters:
        N (int): number of bootstrap iterations.
    """

    dgm = M.persistence_diagram
    num_pts, distribution = len(M.data), []
    for bootstrap_id in range(N):

        print(str(bootstrap_id) + "th iteration")

        # Randomly select points
        idxs = np.random.choice(num_pts, size=num_pts, replace=True)
        Xboot = M.data[idxs,:] if M.input_type == "point cloud" else M.data[idxs,:][:,idxs]
        f_boot, c_boot = M.filters[idxs,:], M.colors[idxs,:]
        Mboot = M.__class__(filters=f_boot, filter_bnds=M.filter_bnds, colors=c_boot, 
                            resolutions=M.resolutions, gains=M.gains, 
                            input_type=M.input_type, clustering=M.clustering).fit(Xboot)

        # Compute the corresponding persistence diagrams
        dgm_boot, _ = compute_topological_features(Mboot)

        # Compute the bottleneck distance
        npts, npts_boot = len(dgm), len(dgm_boot)
        D1 = np.array([[dgm[pt][1][0], dgm[pt][1][1]] for pt in range(npts)]) 
        D2 = np.array([[dgm_boot[pt][1][0], dgm_boot[pt][1][1]] for pt in range(npts_boot)])
        bottle = bottleneck_distance(D1, D2)
        distribution.append(bottle)
        M.distribution = np.sort(distribution)

def get_distance_from_confidence_level(M, alpha=.95, complex_type='mapper'):
    """
    Compute the bottleneck distance threshold corresponding to a specific confidence level.
    Parameters:
        alpha (float): confidence level.
    Returns:
        distance value (float); each feature whose size is above this distance is sure at confidence level alpha.
    """
    return M.distribution[int(alpha*len(M.distribution))]

def get_confidence_level_from_distance(M, distance):
    """
    Compute the confidence level of a specific bottleneck distance threshold.
    Parameters:
        distance (float): bottleneck distance threshold.
    Returns:
        confidence level (float); each feature whose size is above the distance threshold is sure at this confidence level.
    """
    return len(np.argwhere(M.distribution <= distance))/len(M.distribution)

def get_pvalue(M):
    """
    Compute the p-value, i.e. the opposite of the confidence level of the largest bottleneck distance preserving the topological features.
    Returns:
        p-value (float)
    """
    distancemin = min([np.abs(pt[1][0]-pt[1][1]) for pt in M.persistence_diagram])
    return 1.-M.compute_confidence_from_distance(distancemin)

def compute_differential_coordinates(M, nodes=None, features=None, sparse=False):
    """
    Compute the coordinates that best explain a set of nodes VS the rest of the nodes (in the 1-skeleton of the cover complex) with a Kolmogorov-Smirnov test. Only works if input_type is "point cloud".
    Parameters:
        nodes (list of integers): list of nodes to try. For instance, one can take the list of nodes obtained after calling "compute_topological_features"
        features (list of integers): the coordinates to try. All coordinates are tested if None.
        sparse (bool): set to True if your data is sparse and there will be speedup, otherwise use False.
    Returns:
        features (list of integers): the list of coordinates, ranked from smallest to largest p-values.
        p-values (list of float): the corresponding p-values. 
    """
    if M.input_type == "distance matrix":
        print("Need coordinates for running differential coordinates!")
        raise

    node_info = M.node_info
    X = M.data
    nodes = [s[0] for s,_ in self.simplex_tree.get_skeleton(0)] if nodes is None else nodes

    if features is None:
        features = np.arange(X.shape[1])

    list_idxs1 = list(np.unique(np.concatenate([node_info[node_name]["indices"] for node_name in nodes])))
    list_idxs2 = list(set(np.arange(X.shape[0]))-set(list_idxs1))
    pvals = []
    for f in features:
        if sparse:
            Xsp = csr_matrix(X)
            group1 = np.squeeze(np.array(Xsp[list_idxs1,f].todense()))
            group2 = np.squeeze(np.array(Xsp[list_idxs2,f].todense()))
        else:
            group1, group2 = X[list_idxs1,f], X[list_idxs2,f]
        _,pval = ks_2samp(group1, group2)
        pvals.append(pval)
    pvals = np.array(pvals)
    F, P = features[np.argsort(pvals)], np.sort(pvals) 
    return F, P

There are various postprocessing one can do on a cover complex. For instance, one can compute the topological features in the complex. For our human shape, the topological features (identified by computing the persistence of the color function on the complex) are the three branches corresponding to the arms and legs (the lower leg correspond to the whole connected component).

In [ ]:
dgm, bnd = compute_topological_features(cover_complex, threshold=0.)

In [ ]:
G = cover_complex.get_networkx()
plt.figure(figsize=(8,2))
for idx, bd in enumerate(bnd):
    plt.subplot(1,len(bnd),idx+1)
    nx.draw(G, pos=nx.kamada_kawai_layout(G), 
            node_color=[1 if node in bd else 0 for node in G.nodes()], node_size=5)
plt.show()

You can also identify the robust topological features by bootstrapping, and select those associated to 95% confidence.

In [ ]:
bootstrap_topological_features(cover_complex, 100)

In [ ]:
dist = get_distance_from_confidence_level(cover_complex, .95)

In [ ]:
bnd_boot = [b for idx, b in enumerate(bnd) if np.abs(.5 * (dgm[idx][1][1]-dgm[idx][1][0])) >= dist]

In [ ]:
G = cover_complex.get_networkx()
plt.figure(figsize=(8,2))
for idx, bd in enumerate(bnd_boot):
    plt.subplot(1,len(bnd_boot),idx+1)
    nx.draw(G, pos=nx.kamada_kawai_layout(G), 
            node_color=[1 if node in bd else 0 for node in G.nodes()], node_size=5)
plt.show()

Finally, one can identify the coordinates that best explain a topological feature VS the rest of the complex with a Kolmogorov-Smirnov test. In particular, for each topological feature, we can rank the coordinates with respect to their p-values. 

For instance, coordinate 2 (height) is the one that best distinguishes the leg from the rest.

In [ ]:
compute_differential_coordinates(cover_complex, nodes=bnd[1])

On the other hand, coordinate 0 best explains both arms.

In [ ]:
compute_differential_coordinates(cover_complex, nodes=bnd[2])

In [ ]:
compute_differential_coordinates(cover_complex, nodes=bnd[3])